<a href="https://colab.research.google.com/github/gcmillar/CEUS/blob/master/gcmillar_CEUS_Space_time_analytics_JupyterNB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

---
---
#        **Space-time Analytics of Human Physiology for Urban Planning**
---
---

This is code and data for computational reproducibility of the following paper:
> **Garrett C. Millar**, Ondrej Mitas, Wilco Boode Lisette Hoeke, Joost de Kruijff, Anna Petrasova, Helena Mitasova (2020): 
Space-time Analytics of Human Physiology for Urban Planning. In: Computers, Environment, and Urban Sytstems, In: *Advances in portable sensing methodologies for urban environments: Understanding cities from a mobility perspective*. 
[DOI here]

<center>
  <img src="https://raw.githubusercontent.com/gcmillar/CEUS/master/img/viewshed_progress_onedir_ortho.png" alt="runbutton" height="200px">
</center>


## Table of Contents

In [3]:
!jupyter toc2

Traceback (most recent call last):
  File "/Library/Frameworks/Python.framework/Versions/3.8/bin/jupyter", line 10, in <module>
    sys.exit(main())
  File "/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/jupyter_core/command.py", line 247, in main
    command = _jupyter_abspath(subcommand)
  File "/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/jupyter_core/command.py", line 133, in _jupyter_abspath
    raise Exception(
Exception: Jupyter command `jupyter-toc2` not found.


>[Space-time Analytics of Human Physiology for Urban Planning](#scrollTo=hcqShGttHsb2)

>>[Table of Contents](#scrollTo=5Xi0WE1VA7oz)

>>[Code](#scrollTo=uLnyFCClJn6F)

>>[Things to Know](#scrollTo=-i2K8lJ4J1z4)

>>[Setup & Imports](#scrollTo=eRwEVbK3LaXX)

>>>[GDAL & Colaboratory Notebooks](#scrollTo=3UewP07YeyjZ)

>>>>[Install GDAL in R](#scrollTo=AvGtTNhMfhVU)

>[Software: R](#scrollTo=PZk_VvpuKf0F)

>>>[Install Required R Packages](#scrollTo=n9t-PhSneJ-H)

>>[Data Processing](#scrollTo=2OV72uepLixb)

>>>[Artifact Correction](#scrollTo=eTFxJNzJLufO)

>>>[Joining](#scrollTo=iUlGnMdvYSko)

>>>[Identifying Cycle Highway Directions](#scrollTo=ZaUNk_NKhMgT)

>>>[Skin Conductance](#scrollTo=ZWLfza5-Yf3l)

>[Spatial Data](#scrollTo=tI2tuAZEbvWQ)

>>[Processing & Transformation](#scrollTo=Msp57oPXJ8R6)

>>[Processing & Export for Interactive Application - Stress3d](#scrollTo=SWFB0G1uYEf3)

>>[Exporting as Vector to GRASS](#scrollTo=MbZGnHMkYEf0)

>[Software: GRASS GIS](#scrollTo=oC2lpaBs3Q8D)

>>[Install](#scrollTo=vSy1KNtghIxl)

>>[CEUS Data — GRASS GIS](#scrollTo=DExWI9fwO6Lr)

>>>>[Elevation](#scrollTo=DExWI9fwO6Lr)

>>>>[Land Use](#scrollTo=DExWI9fwO6Lr)

>>>[Download CEUS Location Dataset](#scrollTo=-ydwDKOAhZzc)

>>[Configure Python to Run GRASS](#scrollTo=sLCc64a_z-rv)

>[Analyses](#scrollTo=Qgy56fCTrx0L)

>>[Distance to Roads](#scrollTo=9k6luw1l6xAJ)

>>[Viewshed Analysis](#scrollTo=TB4_hakk5Zqi)

>>>[Parallelization in Python](#scrollTo=JSIi-rm0FkI8)

>>[Viewshed Results](#scrollTo=T-fBJIJEDiuU)

>>[Final Export of Data](#scrollTo=KfINmNCEZGw8)

>>[Statistical Modeling](#scrollTo=UEOfNFaqG1WM)



[Clickable_visible_hyperlink](Hidden_landing_URL)

# Code
This is a modified version of a notebook for reproducibility of the applied methodological framework applied in the Millar et al. 2020 paper. The original notebook is at https://colab.research.google.com/github/gcmillar/CEUS/blob/master/PeopleEnvironmentGrassGIS.ipynb

The tools implementing the published methods are available at https://grass.osgeo.org/grass7/manuals/addons/.

<br/>

---

<br/>

<table class="tfo-notebook-buttons" align="center" text-decoration="none">
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/gcmillar/CEUS/blob/master/gcmillar_CEUS_Space_time_analytics_JupyterNB.ipynb
    "><img src="https://raw.githubusercontent.com/gcmillar/CEUS/master/img/colab_logo_32px.png" />Run in Google Colab</a>&nbsp;&nbsp;&nbsp;
  </td>
  <td>
    <a target="_blank" href="https://github.com/gcmillar/CEUS/blob/master/gcmillar_CEUS_Space_time_analytics_JupyterNB.ipynb"><img src="https://raw.githubusercontent.com/gcmillar/CEUS/master/img/GitHub-Mark-26px.png" />View source on GitHub</a>
  </td>
    <td>
    <a href="https://drive.google.com/a/ncsu.edu/uc?authuser=0&id=1L7p5i_xBsYkw1Dav0iiDvahSuxKCaJ0i&export=download"><img src="https://raw.githubusercontent.com/gcmillar/CEUS/master/img/download_logo_32px.png" />Download notebook</a>
  </td>
</table>
<br/>


---

## Things to Know 
* Click play button in upper left corner of code window to run script: <img src="https://raw.githubusercontent.com/gcmillar/CEUS/master/img/codebutton.png" alt="runbutton" width="14" height="14">
<!-- ![alt text](https://raw.githubusercontent.com/gcmillar/CEUS/master/img/codebutton.png "Logo Title Text 1") -->

* Code cells with long computational times: <img src="https://raw.githubusercontent.com/gcmillar/CEUS/master/img/wait_time.png" alt="runbutton" width="25" height="25">

---


## Setup & Imports

%%capture
# !jupyter nbextension enable src/jupyter_contrib_nbextensions/nbextensions/toc2
!find / -name jupyter_contrib_nbextensions*

%%capture
!jupyter nbextension install --user https://rawgithub.com/minrk/ipython_extensions/master/nbextensions/toc.js
!curl -L https://rawgithub.com/minrk/ipython_extensions/master/nbextensions/toc.css > $(jupyter --data-dir)/nbextensions/toc.css
!jupyter nbextension enable toc2;

import warnings
warnings.filterwarnings(action='once')
# warnings.filterwarnings('ignore')

###R
#### R Magic

Very needed step, load cell magic for R, where then each R-script cell is begun with: `%%R`  

%%capture
# activate R magic
%load_ext rpy2.ipython

#### Install GDAL in R

%%capture
%%R
install.packages("rgdal")
library(rgdal);

### GDAL & Colaboratory Notebooks 

Need to setup the colaboratory notebook with GDAL, to be used in R. As this will be running on a ubuntu server, run 
*   `apt-get update`
*   `apt-get install libgdal-dev` 
    *   run this before trying to install GDAL otherwise you will get errors.
    * The flag -y accepts the download and runs – this is the longest part (but not too long)
*   Install python-gdal again with the -y flag
*   Install NumPy and SciPy (not compulsory, but why not?)
*   Test the installation by importing GDAL. No error? It should be ok.

> **NOTE**: <img src="https://raw.githubusercontent.com/gcmillar/CEUS/master/img/wait_time.png" alt="runbutton" width="25" height="25">

%%capture
#Step 1
!apt-get update
#Step 2
!apt-get install libgdal-dev -y
#Step 3
!apt-get install python-gdal -y
#Step 4
!apt-get install python-numpy python-scipy -y
#Step 5
import gdal 

### Pandas

%%capture
import pandas as pd

# Software: **R**

---

### Install Required R Packages 
> **NOTE**: <img src="https://raw.githubusercontent.com/gcmillar/CEUS/master/img/wait_time.png" alt="runbutton" width="25" height="25">

%%capture
%%R
options(digits = 8)
if (!require(dplyr)) {install.packages("dplyr"); library(dplyr)}
if (!require(tidyr)) {install.packages("tidyr"); library(tidyr)}
if (!require(sp)) {install.packages("sp"); library(sp)}
if (!require(raster)) {install.packages("raster"); library(raster)}
if (!require(rgeos)) {install.packages("rgeos"); library(rgeos)}
if (!require(rgbif)) {install.packages("rgbif"); library(rgbif)}
if (!require(viridis)) {install.packages("viridis"); library(viridis)}
if (!require(gridExtra)) {install.packages("gridExtra"); library(gridExtra)}
if (!require(R.matlab)) {install.packages("R.matlab"); library(R.matlab)}
if (!require(plyr)) {install.packages("plyr"); library(plyr)}
if (!require(data.table)) {install.packages("data.table"); library(data.table)}
if (!require(maptools)) {install.packages("maptools"); library(maptools)}
if (!require(spatstat)) {install.packages("spatstat"); library(spatstat)}
if (!require(rasterVis)) {install.packages("rasterVis"); library(rasterVis)}
if (!require(leaflet)) {install.packages("leaflet"); library(leaflet)}
if (!require(RColorBrewer)) {install.packages("RColorBrewer"); library(RColorBrewer)}
if (!require(scales)) {install.packages("scales"); library(scales)}
if (!require(spgwr)) {install.packages("spgwr"); library(spgwr)}
if (!require(gdalUtils)) {install.packages("gdalUtils"); library(gdalUtils)}
if (!require(devtools)) {install.packages("devtools"); library(devtools)}
if (!require(tidyverse)) {install.packages("tidyverse"); library(tidyverse)}
if (!require(sf)) {install.packages("sf"); library(sf)}
if (!require(ggmap)) {install.packages("ggmap"); library(ggmap)}
if (!require(formatR)) {install.packages("formatR"); library(formatR)}
if (!require(stats)) {install.packages("stats"); library(stats)}
if (!require(data.table)) {install.packages("data.table"); library(data.table)}
if (!require(lubridate)) {install.packages("lubridate"); library(lubridate)}
if (!require(ggplot2)) {install.packages("ggplot2"); library(ggplot2)}
if (!require(grid)) {install.packages("grid"); library(grid)}
if (!require(gridExtra)) {install.packages("gridExtra"); library(gridExtra)}
if (!require(lattice)) {install.packages("lattice"); library(lattice)}
if (!require(leaflet)) {install.packages("leaflet"); library(leaflet)}
if (!require(lme4)) {install.packages("lme4"); library(lme4)}
if (!require(corrgram)) {install.packages("corrgram"); library(corrgram)}
if (!require(psych)) {install.packages("psych"); library(psych)};

## Data Processing

Read in CEUS Initial Cyclist Data [.csv]; using [online link from GitHub](https://raw.githubusercontent.com/gcmillar/CEUS/master/CHIPS_all.csv):

> **File Description**: Initial dataset for CEUS analysis (.csv of cyclists' location and physiological data) 

> **Note**: This data is read in under the assumption the raw skin conductance signals have been properly processed in Ledalab--MATLAB-based software for the analysis of raw EDA data. This process used a moving window of 20 seconds to identify deviations of 3 standard deviations or more as potential motion artifacts. Deviations were visually inspected and if they failed to conform to a standard, physiologically-plausible shape for a SCR, were replaced with linear interpolation. Cleaned data were then deconvoluted using the Ledalab toolbox into phasic and tonic components. After deconvolution, we can now proceed with the data representing the phasic component, where the physiological data are georeferenced through a time-based synchronization of the SCR data with GPS location data.
<!-- ~~~ -->
 <!-- **File Description**: Initial dataset for CEUS analysis (.csv of cyclists' location and physiological data)  -->
<!-- ~~~ -->


%%capture
# import csv
!wget "https://raw.githubusercontent.com/gcmillar/CEUS/master/CHIPS_all.csv"
CHIPS_url = 'https://raw.githubusercontent.com/gcmillar/CEUS/master/CHIPS_all.csv'
CHIPS_df = pd.read_csv(CHIPS_url) # Dataset is now stored in a Pandas Dataframe

Using Pandas to display CHIPS cycling data as interactive table.

%%capture
%load_ext google.colab.data_table
CHIPS_df

# # @interact
# def show_articles_more_than(column='participant', x=5000):
#     return df.loc[df[column] > x]

# import ipywidgets as widgets
# from ipywidgets import interact, interact_manual

# @interact
# def show_articles_more_than(column='participant', x=5000):
#     return CHIPS_df.loc[CHIPS_df[column] > x]

%%capture
%unload_ext google.colab.data_table

%%R
setwd("/content/")
CHIPS_df <- read.table("CHIPS_all.csv", header = TRUE, row.names=NULL, sep=",")
setnames(CHIPS_df, "long", "lon")

### Artifact Correction
The script and descriptions below detail the process of removing the first and last 4.00% of the raw skin conductance data to correct for artifacts: 
Using time, we know 4.00% of `max(CHIPS_df$time)` is `max(CHIPS_df$time) * 0.040` seconds. So, we'll delete the first and last `max(CHIPS_df$time) * 0.040` seconds (or `max(CHIPS_df$time) * 0.040 / 60` 
minutes) in the data to remove the observed artifacts. Since the data was collected every 0.25 seconds, that means `round(max(CHIPS_df$time) * 0.040 * 4)` rows should be deleted at the beginning and end of the data set.

%%R
unique(CHIPS_df$participant)
# keeping specific pt data
CHIPS_pts <- CHIPS_df
# data needs to first be separated by participants to remove artifacts 
pt1 <- CHIPS_pts[CHIPS_pts$participant == 1, ]
pt2 <- CHIPS_pts[CHIPS_pts$participant == 2, ]
pt3 <- CHIPS_pts[CHIPS_pts$participant == 3, ]
pt4 <- CHIPS_pts[CHIPS_pts$participant == 4, ]
pt5 <- CHIPS_pts[CHIPS_pts$participant == 5, ]
pt6 <- CHIPS_pts[CHIPS_pts$participant == 6, ]
pt7 <- CHIPS_pts[CHIPS_pts$participant == 7, ]
pt8 <- CHIPS_pts[CHIPS_pts$participant == 8, ]
pt9 <- CHIPS_pts[CHIPS_pts$participant == 9, ]
pt10 <- CHIPS_pts[CHIPS_pts$participant == 10, ]
pt11 <- CHIPS_pts[CHIPS_pts$participant == 11, ]
pt12 <- CHIPS_pts[CHIPS_pts$participant == 12, ]
# first rows
N <- round(max(CHIPS_pts$time) * 0.040 * 4)
pt1 <- pt1[-(1:N), , drop = FALSE]
pt2 <- pt2[-(1:N), , drop = FALSE]
pt3 <- pt3[-(1:N), , drop = FALSE]
pt4 <- pt4[-(1:N), , drop = FALSE]
pt5 <- pt5[-(1:N), , drop = FALSE]
pt6 <- pt6[-(1:N), , drop = FALSE]
pt7 <- pt7[-(1:N), , drop = FALSE]
pt8 <- pt8[-(1:N), , drop = FALSE]
pt9 <- pt9[-(1:N), , drop = FALSE]
pt10 <- pt10[-(1:N), , drop = FALSE]
pt11 <- pt11[-(1:N), , drop = FALSE]
pt12 <- pt12[-(1:N), , drop = FALSE]
# last rows
N.2 <- round(max(CHIPS_pts$time) * 0.040 * 4)
pt1 <- pt1[1:(nrow(pt1)-N.2),]
pt2 <- pt2[1:(nrow(pt2)-N.2),]
pt3 <- pt3[1:(nrow(pt3)-N.2),]
pt4 <- pt4[1:(nrow(pt4)-N.2),]
pt5 <- pt5[1:(nrow(pt5)-N.2),]
pt6 <- pt6[1:(nrow(pt6)-N.2),]
pt7 <- pt7[1:(nrow(pt7)-N.2),]
pt8 <- pt8[1:(nrow(pt8)-N.2),]
pt9 <- pt9[1:(nrow(pt9)-N.2),]
pt10 <- pt10[1:(nrow(pt10)-N.2),]
pt11 <- pt11[1:(nrow(pt11)-N.2),]
pt12 <- pt12[1:(nrow(pt12)-N.2),]

### Joining 
All participant data that were just separated and cleaned by using time are re-included into the same data frame.

%%R
# join
CHIPS_df_clean <- rbind(pt1, pt2, pt3, pt4, pt5, pt6, pt7, pt8, pt9, pt10, pt11, pt12)
CHIPS_df_clean <- add_count(CHIPS_df_clean, lon, lat, name = "data.points.n")
CHIPS_df_clean$lon.lat <- paste(CHIPS_df_clean$lon, CHIPS_df_clean$lat, sep=",")
CHIPS_time.in.cell <- aggregate(time ~ lat + lon, data = CHIPS_df_clean, sum)

### Identifying Cycle Highway Directions

<!-- <mark>**Should this code be included, since the resulting charts didn't end up making the paper?**</mark> -->

Half participants went Waalwijk --> Tilburg, other half went Tilburg --> Waalwijk:
*   `Waalwijk --> Tilburg: 1, 3, 4, 10, 11, 12`
*   `Tilburg --> Waalwijk: 2, 5, 6, 7, 8, 9`

> Groups need to be split for proper GPS data visualiztion

%%R
CHIPS_df_clean$direction <- NA
CHIPS_df_clean$direction[which(CHIPS_df_clean$participant %in% c(1, 3, 4, 10, 11, 12))] <- "waal_til"
CHIPS_df_clean$direction[which(CHIPS_df_clean$participant %in% c(2, 5, 6, 7, 8, 9))] <- "til_waal"
CHIPS_df_clean$direction <- as.factor(CHIPS_df_clean$direction)
CHIPS_waal_til <- subset(CHIPS_df_clean, direction=="waal_til")
CHIPS_til_waal <- subset(CHIPS_df_clean, direction=="til_waal")

### Skin Conductance

<mark>**Should this code be included, since the resulting charts didn't end up making the paper?**</mark>

Plotting SCR data as a function of elevation and speed by time stamps of each way point using `ggplot`. This helps examine examine potentially confounding effects between variables.

As to not confuse ggplot, we do not pass the `SpatialPointsDataFrame`-object directly, but convert it to a regular data frame with `as.data.frame()`:

%%R
# restricting decimal places for graphs
options(digits = 0)
# plot of time and elevation, colored by skin conductance
# CHIPS_waal_til
time_elev_conduct_plot.waal_til <- ggplot(as.data.frame(CHIPS_waal_til), # convert to regular dataframe
                                          aes(x=time, y=altitude, color = phasic_z)) +
  scale_color_gradient2(low= "dodgerblue4", mid="slategray2", high="red3",
                        midpoint = median(CHIPS_df$phasic_z),
                        breaks=seq(min(CHIPS_df$phasic_z), max(CHIPS_df$phasic_z),
                                    (max(CHIPS_df$phasic_z)-min(CHIPS_df$phasic_z))/4)) +
  theme(panel.grid.major = element_line(colour = "#808080"),
        legend.title = element_blank()) +
  theme_linedraw(base_size = 28) +
  geom_point(alpha = 0.6, size = 3.5, show.legend = FALSE) + 
  geom_smooth(span = 0.3, colour="#90CA77", size=3, alpha=0.6) +
   # theme_bw() + 
   ggtitle("Waalwijk to Tilburg") +
  labs(x='\nCycling time', y='')

# CHIPS_til_waal
CHIPS_til_waal[order(CHIPS_til_waal$distance),]
CHIPS_til_waal$participant[1:50]
time_elev_conduct_plot.til_waal <- ggplot(as.data.frame(CHIPS_til_waal), # convert to regular dataframe
            aes(x=time, y=altitude, color = phasic_z)) +
  scale_color_gradient2(low= "dodgerblue4", mid="slategray2", high="red3", 
                       midpoint = median(CHIPS_df$phasic_z),
                       breaks=seq(min(CHIPS_df$phasic_z), max(CHIPS_df$phasic_z),
                                    (max(CHIPS_df$phasic_z)-min(CHIPS_df$phasic_z))/4)) +
  theme(panel.grid.major = element_line(colour = "#808080")) +
  theme_linedraw(base_size = 28) +
  geom_point(alpha = 0.6, size = 3.5, show.legend = FALSE) + 
  geom_smooth(span = 0.3, colour="#90CA77", size=3, alpha=0.6) +
  ggtitle("Tilburg to Waalwijk") +
  labs(x='\nCycling time', y='')
# together
grid.arrange(time_elev_conduct_plot.waal_til, time_elev_conduct_plot.til_waal, ncol=2)

# plot of time and speed, colored by skin conductance
# CHIPS_waal_til

# subset(CHIPS_waal_til, CHIPS_waal_til$participant==9,CHIPS_waal_til$distance>100)

time_speed_conduct_plot.waal_til <- ggplot(as.data.frame(CHIPS_waal_til), # convert to regular dataframe
                                          aes(x=time, y=speed, color = phasic_z)) +
  geom_smooth(span = 0.3, colour="darkgray", size=3, alpha=0.6) +
  scale_color_gradient2(low= "dodgerblue4", mid="slategray2", high="red3", 
                        midpoint = median(CHIPS_df$phasic_z),
                        breaks=seq(min(CHIPS_df$phasic_z), max(CHIPS_df$phasic_z),
                                    (max(CHIPS_df$phasic_z)-min(CHIPS_df$phasic_z))/4)) +
  theme(panel.grid.major = element_line(colour = "#808080"),
        legend.title = element_blank()) +
  theme_linedraw(base_size = 28) +
  geom_point(alpha = 0.6, size = 3.5, show.legend = FALSE) + 
  geom_smooth(span = 0.3, colour="#90CA77", size=3, alpha=0.6) +
   # theme_bw() + 
   ggtitle("Waalwijk to Tilburg") +
  labs(x='\nCycling time', y='')
# CHIPS_til_waal
time_speed_conduct_plot.til_waal <- ggplot(as.data.frame(CHIPS_til_waal), # convert to regular dataframe
            aes(x=time, y=speed, color = phasic_z)) +
  geom_smooth(span = 0.3, colour="darkgray", size=3, alpha=0.6) +
  scale_color_gradient2(low= "dodgerblue4", mid="slategray2", high="red3", 
                       midpoint = median(CHIPS_df$phasic_z),
                       breaks=seq(min(CHIPS_df$phasic_z), max(CHIPS_df$phasic_z),
                                    (max(CHIPS_df$phasic_z)-min(CHIPS_df$phasic_z))/4)) +
  theme(panel.grid.major = element_line(colour = "#808080")) +
  theme_linedraw(base_size = 28) +
  geom_point(alpha = 0.6, size = 3.5, show.legend = FALSE) + 
  geom_smooth(span = 0.3, colour="#90CA77", size=3, alpha=0.6) +
  ggtitle("Tilburg to Waalwijk") +
  labs(x='\nCycling time', y='')
# together
grid.arrange(time_speed_conduct_plot.waal_til, time_speed_conduct_plot.til_waal, ncol=2)


# direction instead of time on x axis
# first elevation
# order_by(CHIPS_waal_til$altitude, CHIPS_waal_til$distance)
CHIPS_waal_til <- subset(CHIPS_waal_til, distance>=100)
dist_elev_conduct_plot.waal_til <- ggplot(as.data.frame(CHIPS_waal_til), # convert to regular dataframe
                                          aes(x=distance, y=altitude, color = phasic_z)) +
  scale_color_gradient2(low= "dodgerblue4", mid="slategray2", high="red3",
                        midpoint = median(CHIPS_df$phasic_z),
                        breaks=seq(min(CHIPS_df$phasic_z), max(CHIPS_df$phasic_z),
                                    (max(CHIPS_df$phasic_z)-min(CHIPS_df$phasic_z))/4)) +
  theme(panel.grid.major = element_line(colour = "#808080"),
        legend.title = element_blank()) +
  theme_linedraw(base_size = 28) +
  geom_point(alpha = 0.6, size = 3.5, show.legend = FALSE) + 
  geom_smooth(span = 0.3, colour="#90CA77", size=3, alpha=0.6) +
   # theme_bw() + 
   ggtitle("Waalwijk to Tilburg") +
  labs(x='\nDistance (m)', y='')

# CHIPS_til_waal
CHIPS_til_waal <- subset(CHIPS_til_waal, distance>=100)
dist_elev_conduct_plot.til_waal <- ggplot(as.data.frame(CHIPS_til_waal), # convert to regular dataframe
            aes(x=distance, y=altitude, color = phasic_z)) +
  scale_color_gradient2(low= "dodgerblue4", mid="slategray2", high="red3", 
                       midpoint = median(CHIPS_df$phasic_z),
                       breaks=seq(min(CHIPS_df$phasic_z), max(CHIPS_df$phasic_z),
                                    (max(CHIPS_df$phasic_z)-min(CHIPS_df$phasic_z))/4)) +
  theme(panel.grid.major = element_line(colour = "#808080")) +
  theme_linedraw(base_size = 28) +
  geom_point(alpha = 0.6, size = 3.5, show.legend = FALSE) + 
  geom_smooth(span = 0.3, colour="#90CA77", size=3, alpha=0.6) +
  ggtitle("Tilburg to Waalwijk") +
  labs(x='\nDistance (m)', y='')
# together
grid.arrange(dist_elev_conduct_plot.waal_til,dist_elev_conduct_plot.til_waal, ncol=2)


# now for speed

dist_speed_conduct_plot.waal_til <- ggplot(as.data.frame(CHIPS_waal_til), # convert to regular dataframe
                                          aes(x=distance, y=speed, color = phasic_z)) +
  scale_color_gradient2(low= "dodgerblue4", mid="slategray2", high="red3",
                        midpoint = median(CHIPS_df$phasic_z),
                        breaks=seq(min(CHIPS_df$phasic_z), max(CHIPS_df$phasic_z),
                                    (max(CHIPS_df$phasic_z)-min(CHIPS_df$phasic_z))/4)) +
  theme(panel.grid.major = element_line(colour = "#808080"),
        legend.title = element_blank()) +
  theme_linedraw(base_size = 28) +
  geom_point(alpha = 0.6, size = 3.5, show.legend = FALSE) + 
  geom_point() + 
  geom_smooth(span = 0.3, colour="#90CA77", size=3, alpha=0.6) +
   # theme_bw() + 
   ggtitle("Waalwijk to Tilburg") +
  labs(x='\nDistance (m)', y='')

# CHIPS_til_waal
CHIPS_til_waal <- subset(CHIPS_til_waal, speed>0)
dist_speed_conduct_plot.til_waal <- ggplot(as.data.frame(CHIPS_til_waal), # convert to regular dataframe
            aes(x=distance, y=speed, color = phasic_z)) +
  scale_color_gradient2(low= "dodgerblue4", mid="slategray2", high="red3", 
                       midpoint = median(CHIPS_df$phasic_z),
                       breaks=seq(min(CHIPS_df$phasic_z), max(CHIPS_df$phasic_z),
                                    (max(CHIPS_df$phasic_z)-min(CHIPS_df$phasic_z))/4)) +
  theme(panel.grid.major = element_line(colour = "#808080")) +
  theme_linedraw(base_size = 28) +
  geom_point(alpha = 0.6, size = 3.5, show.legend = FALSE) + 
  geom_smooth(span = 0.3, colour="#90CA77", size=3, alpha=0.6) +
  ggtitle("Tilburg to Waalwijk") +
  labs(x='\nDistance (m)', y='')
# together
grid.arrange(dist_speed_conduct_plot.waal_til,dist_speed_conduct_plot.til_waal, ncol=2)

# Spatial Data 
For spatial analyses to occur, the data frame needs to be converted into a spatial data frame. The functions `coordinates()` and `projection()` from `sp` library converts the dataset into spatial objects that our to-be-conducted analyses requires. 

In particular, `coordinates()` specifies the latitude and longitude of the data and `proj4string()` creates the projection layer, i.e. the coordinate system. As you know, the EPSG: 4326 means that the coordinates are latitude-longitude pairs on a reference ellipsoid given by WGS84, projected with Mercator Projection.


## Processing & Transformation

%%R
# Data Processing
CHIPS_lonlat_sp <- CHIPS_df_clean
# First, ensuring a default long / lat is assigned to the coordinates:
## Specifying coordinates in raw data
coordinates(CHIPS_lonlat_sp) <- ~ lon + lat
## Setting default projection
proj4string(CHIPS_lonlat_sp) <- CRS("+init=epsg:4326 +proj=longlat +ellps=WGS84 +datum=WGS84 +no_defs +towgs84=0,0,0")
# Then, transform the coordinates to UTM, using its proper CRS [EPSG:28992; RD / Amersfoort].
## Transforming coordinates to UTM
CHIPS_sp <- spTransform(CHIPS_lonlat_sp, "+proj=sterea +lat_0=52.15616055555555 +lon_0=5.38763888888889 +k=0.9999079 +x_0=155000 +y_0=463000 +ellps=bessel +units=m +no_defs")
# Creating specific participant data frames
pt1 <- CHIPS_sp[CHIPS_sp$participant == 1, ]
pt2 <- CHIPS_sp[CHIPS_sp$participant == 2, ]
pt3 <- CHIPS_sp[CHIPS_sp$participant == 3, ]
pt4 <- CHIPS_sp[CHIPS_sp$participant == 4, ]
pt5 <- CHIPS_sp[CHIPS_sp$participant == 5, ]
pt6 <- CHIPS_sp[CHIPS_sp$participant == 6, ]
pt7 <- CHIPS_sp[CHIPS_sp$participant == 7, ]
pt8 <- CHIPS_sp[CHIPS_sp$participant == 8, ]
pt9 <- CHIPS_sp[CHIPS_sp$participant == 9, ]
pt10 <- CHIPS_sp[CHIPS_sp$participant == 10, ]
pt11 <- CHIPS_sp[CHIPS_sp$participant == 11, ]
pt12 <- CHIPS_sp[CHIPS_sp$participant == 12, ]
# inspect resulting data
print(CHIPS_lonlat_sp)

## Processing & Export for Interactive Application - Stress3d

Using the same approach above for the web-mapping application, Stress3d. Mapbox, and web-mapping in general, requires standard latitude and longitude coordinates [EPSG:4326; WGS 84]. Exporting to GeoJSON format, also usually preferred by web-mapping applications.

%%R
CHIPS_lonlat <- CHIPS_df
coordinates(CHIPS_lonlat) <- ~ lon + lat
# Setting default projection
proj4string(CHIPS_lonlat) <- CRS('+proj=longlat +lat_0=52 +lon_0=10 +x_0=4321000 +y_0=3210000 +ellps=GRS80 +units=m +no_defs')
pt1 <- CHIPS_lonlat[CHIPS_lonlat$participant == 1, ]
pt2 <- CHIPS_lonlat[CHIPS_lonlat$participant == 2, ]
pt3 <- CHIPS_lonlat[CHIPS_lonlat$participant == 3, ]
pt4 <- CHIPS_lonlat[CHIPS_lonlat$participant == 4, ]
pt5 <- CHIPS_lonlat[CHIPS_lonlat$participant == 5, ]
pt6 <- CHIPS_lonlat[CHIPS_lonlat$participant == 6, ]
pt7 <- CHIPS_lonlat[CHIPS_lonlat$participant == 7, ]
pt8 <- CHIPS_lonlat[CHIPS_lonlat$participant == 8, ]
pt9 <- CHIPS_lonlat[CHIPS_lonlat$participant == 9, ]
pt10 <- CHIPS_lonlat[CHIPS_lonlat$participant == 10, ]
pt11 <- CHIPS_lonlat[CHIPS_lonlat$participant == 11, ]
pt12 <- CHIPS_lonlat[CHIPS_lonlat$participant == 12, ]
# Exporting to GeoJSON
writeOGR(CHIPS_lonlat, "/content/CHIPS_all_pts.geojson",
         layer="CHIPS_lonlat", driver="GeoJSON")

import os
import pandas as pd
from mapboxgl.utils import create_color_stops, df_to_geojson
from mapboxgl.viz import CircleViz

# Load data 
df = CHIPS.csv

# Must be a public token, starting with `pk`
token = os.getenv('pk.eyJ1IjoiZ2NtaWxsYXIiLCJhIjoiY2pvcDhrbGl4MDFvaTNrczR0d2hxcjdnNSJ9.JYgBw6y2pEq_AEAOCaoQpw')

# Create a geojson file export from a Pandas dataframe
df_to_geojson(df, filename='points.geojson',
              properties=['Avg Medicare Payments', 'Avg Covered Charges', 'date'],
              lat='lat', lon='lon', precision=3)

# Generate data breaks and color stops from colorBrewer
color_breaks = [0,10,100,1000,10000]
color_stops = create_color_stops(color_breaks, colors='YlGnBu')

# Create the viz from the dataframe
viz = CircleViz('points.geojson',
                access_token=token,
                height='400px',
                color_property = "Avg Medicare Payments",
                color_stops = color_stops,
                center = (-95, 40),
                zoom = 3,
                below_layer = 'waterway-label'
              )
viz.show()

%%html
<iframe src="https://gcmillar.github.io/stress3d/" width="1200" height="800"></iframe>

## Exporting as Vector to GRASS
The following computes a csv files designed to be seamlessly imported into GRASS. Coordinates are in UTM [EPSG:28992; RD / Amersfoort] format, csv separator is set to pipe ("|" - GRASS's default import setting).  

%%R
# ALL
chips_all <- CHIPS_sp@data[,1:13]
chips_all$lon <-CHIPS_sp@coords[,1]
chips_all$lat <-CHIPS_sp@coords[,2]
write.table(chips_all, row.names = F, quote = F, 
            col.names = T, sep = "|", dec = ".",
           "/content/chips_all.txt")

---

# Software: **GRASS GIS**

---

## Install 
**Source**<sup> [1](https://grasswiki.osgeo.org/wiki/GRASS_GIS_Jupyter_notebooks)</sup> 

When the following code chunk is run, you will see a rectangular input box pop up below. Click in it using your mouse, and then press 'Enter' to continue the configuration process.

> **NOTE**: <img src="https://raw.githubusercontent.com/gcmillar/CEUS/master/img/wait_time.png" alt="runbutton" width="25" height="25">

%%capture
!add-apt-repository ppa:ubuntugis/ubuntugis-unstable
#update packages
!apt-get update
#install grass
!apt-get install grass -y;

## CEUS Data — GRASS GIS

This dataset contains the already developed digital surface model (DSM; downloaded from the official
open data repository:  
#### *Elevation*
DSM was interpolated from first-return lidar points at half-meter resolution to provide input for computation of viewsheds along the cyclists path while capturing impact of buildings and other structures. We used a regularized
spline with tension algorithm implemented in GRASS GIS to balance the smoothness and approximation accuracy of the surface.
> **Source**: The Actual Elevation map of the Netherlands (AHN)<sup>[1](http://ahn.nl)</sup> 

---

#### *Land Use*
Land use was split into the following seven classes: developed, natural, recreation, water, business, agriculture, and forest. Developed areas were classified as any area with high levels of surrounding urban life. Natural areas consisted of grasslands, herbaceous, unpaved surfaces, grass, orchards, meadows, and any other layers constituted by greenery. Recreation contained places frequented for leisure like parks, amusement parks (specifically Efteling, a fantasy-themed amusement park), soccer pitches, and camp sites. Any area described as containing a body of water was also defined. The polygon-based land use layer was then converted to 0.5 m resolution raster representation.

---

### Download CEUS Location Dataset 

# Download & import CEUS data as GRASS GIS location file 
import zipfile
# The following snippets set the unzipped data as a GRASS location file
!wget /$HOME/grassdata/ "http://fatra.cnr.ncsu.edu/data/CEUS.zip"
zip_ref = zipfile.ZipFile("/content/CEUS.zip", 'r')
zip_ref.extractall("/root/grassdata/")
zip_ref.close()

## Configure Python to Run GRASS



# import Python standard library and IPython packages we need
import os
import sys
import subprocess
from IPython.display import Image

# ask GRASS GIS where its Python packages are
gisbase= "/usr/lib/grass78"
# gisbase = subprocess.check_output(["grass78", "--config", "path"]).strip()
print(gisbase)
os.environ['GISBASE'] = gisbase
sys.path.append(os.path.join("/usr/lib/grass78", "etc", "python"))

# import GRASS GIS packages we need
import grass.script as gs
import grass.script.setup as gsetup

# create GRASS GIS session
rcfile = gsetup.init(gisbase, "/root/grassdata", "CEUS", "PERMANENT")

# we want functions to raise exceptions and see standard output of the modules
gs.set_raise_on_error(True)
gs.set_capture_stderr(True)

# simply overwrite existing maps like we overwrite Python variable values
os.environ['GRASS_OVERWRITE'] = '1'

# enable map rendering to in Jupyter Notebook
os.environ['GRASS_FONT'] = 'sans'

# set display modules to render into a file (named map.png by default)
os.environ['GRASS_RENDER_IMMEDIATE'] = 'cairo'
os.environ['GRASS_RENDER_FILE_READ'] = 'TRUE'
os.environ['GRASS_LEGEND_FILE'] = 'legend.txt'

# Analyses

# import data with selected attributes in EPSG (amersf) location
gs.run_command('v.in.ascii', flags='z', input='/content/chips_all.txt', out='chips_cyclists', separator='|', skip=1, x=14, y=15, z=6, cat=0, columns="participant integer, time double precision, conductance double precision, conductance_z double precision, phasic double precision, phasic_z double precision, tonic double precision, tonic_z double precision, altitude double precision, distance double precision, speed double precision, speed2 double precision, power double precision, lon double precision, lat double precision" )
# check on cyclists stored vector data
gs.parse_command('g.list', type="raster", flags="p")

## Distance to Roads
Since our analyses are interested in any potential dependence between distance from environmental features and associated skin conductance values, we used  [v.distance]<sup>[2](https://grass.osgeo.org/grass79/manuals/v.distance.html)</sup>, a [GRASS GIS]<sup>[3](https://grass.osgeo.org/)</sup> module to map the distances between each cyclist’s locational point and the closest roads. These are then stored for later statistical analyses to determine possible relationships between how far away cyclists were from roads and observed levels of emotional arousal.

> **Note**: Vector modules typically don't follow computational region, but we set it to inform display modules about our area of interest.

gs.parse_command('g.region', n=413730.54450704, s=394620.68549296, w=129168.57323866, e=136079.49676134, nsres=0.99999262, ewres=0.99998893)
# Set resolution (matching DSM res)
gs.run_command('g.region', res=0.5)
# adding necessary columns for resulting output of v.distance [dist_to,lc_type]
gs.run_command('v.db.addcolumn', map='chips_cyclists', columns="dist_to double precision,lc_type varchar")
# same for roads [dist_to_road,road_type]
gs.run_command('v.db.addcolumn', map='chips_cyclists', columns="dist_to_road double precision,road_type varchar")
# find distance from cyclist locations to each specific road type, uploading min distance and road type (large classes: small, med, large)
gs.run_command('v.distance', from_='chips_cyclists@PERMANENT', to='roads_polys', to_type='line,area', upload='dist,to_attr', column='dist_to_road,road_type', to_column='class')
# check attribute table was correctly updated
print(gs.read_command('db.select', table='chips_cyclists', sql='SELECT * FROM chips_cyclists WHERE cat > 1 AND cat < 10'))

## Viewshed Analysis


To initiate the viewshed analytical process, 1739 viewpoints are evenly generated along the cycling highway at 20 m intervals.

This is straightforward to accomplish with GRASS, with use of the v.to.points<sup>[1](https://grass.osgeo.org/grass78/manuals/v.to.points.html)</sup> module. This module creates points along input 2D or 3D lines, boundaries, and faces. Point features including centroids and kernels are copied from input vector map to the output. The `dmax` parameter is of specific interest here, as it is used to set the maximum limit in distance between generated points. 

gs.parse_command('g.list', type="vector", flags="p")
gs.run_command('v.to.points', input='cycle_track_viewpoints', type='point,line,boundary,centroid,area,face,kernel', output='viewpoints_20m', dmax=20)
gs.parse_command('g.list', type="vector", flags="p")
gs.parse_command('v.info', map="viewpoints_20m")

# Download & import raster map data  data as GRASS GIS location file 
!wget /content/ "https://github.com/gcmillar/CEUS/blob/master/cycling_track_dir?raw=true"
!wget /content/ "https://github.com/gcmillar/CEUS/blob/master/cycling_track_dir2?raw=true"
# The following snippets set the unzipped data as a GRASS location file
gs.run_command('r.in.gdal', input='/content/cycling_track_dir?raw=true', output='cycling_track_dir')
gs.run_command('r.in.gdal', input='/content/cycling_track_dir2?raw=true', output='cycling_track_dir2')
gs.parse_command('g.list', type="raster", flags="p")

^^ Created with (actually not true, you gd liar):
```{python}
gs.run_command('v.what.rast', map='viewpoints_20m', raster='cycling_track_dir', column='direction', layer='2')
gs.run_command('v.what.rast', map='viewpoints_20m', raster='cycling_track_dir2', column='direction2', layer=2)
 ```


!wget /content/ "https://github.com/gcmillar/CEUS/blob/master/viewpoints_20m?raw=true"
gs.run_command('v.import', input='/content/viewpoints_20m?raw=true', output='viewpoints_20m')
print(gs.read_command('db.select', table='viewpoints_20m', sql='SELECT * FROM viewpoints_20m WHERE cat > 1 AND cat < 10'))

Then, realizing we need a way to associate the results outputted from the parallelization script, I ran a v.distance module for all viewpoint ranges, to obtain a column that describes the nearest viewpoint at each location point. These of course vary across the three viewpoint ranges (20m, 50m, 100m). I did this approach so that:

gs.run_command('v.db.connect', map='viewpoints_20m', table='viewpoints_20m', layer=2, flags='o')
gs.run_command('v.db.addcolumn', layer=2, map='viewpoints_20m', columns="viewpoint_id_20m double precision")
gs.run_command('v.db.addcolumn', map='chips_cyclists', columns="viewpoint_id_20m double precision")
gs.run_command('v.distance', from_='chips_cyclists', from_type='point', to='viewpoints_20m', to_type='point', upload='cat', column='viewpoint_id_20m', to_column='cat')
print(gs.read_command('db.select', table='viewpoints_20m', sql='SELECT * FROM viewpoints_20m WHERE cat > 1 AND cat < 10'))
print(gs.read_command('db.select', table='chips_cyclists', sql='SELECT * FROM chips_cyclists WHERE cat > 1 AND cat < 10'))

### Parallelization in Python
***At this time, a solution is still needed for running the Python parallelization script (used to generate viewsheds and zonal statistics on visible land cover classes) in a Jupyter Notebook. For the time being, the full Python script is included below, while the following section reads in the .txt output as generated from the successfully-run script below:***


```{python}
#!/usr/bin/env python3

#%module
#% description: Viewshed for CHIPS [CEUS] project
#% keyword: vector
#% keyword: geometry
#%end

#%option G_OPT_M_COORDS
#% key: coordinates
#%end

#%option
#% key: view_id
#% type: integer
#% required: no
#% multiple: no
#% key_desc: value
#% description: View id
#%end

#%option
#% key: direction
#% type: double
#% required: no
#% multiple: no
#% key_desc: value
#% description: Direction
#%end

#%option G_OPT_F_OUTPUT
#% key: output
#%end

import os
import grass.script as gscript

def compute_direction(main_direction):
    half_angle = 90
    mina = main_direction - half_angle
    maxa = main_direction + half_angle
    if maxa > 360:
        maxa -= 360
    if mina < 0:
        mina += 360
    return mina, maxa

def main(elevation, coords, output, pid, direction, sample):
    limit = 1000
    name = 'viewshed'
    gscript.run_command('g.region', raster=elevation)
    gscript.run_command('g.region', n=coords[1] + limit, s=coords[1] - limit,
                            e=coords[0] + limit, w=coords[0] - limit, align=elevation)
    region = gscript.region()
    mina, maxa = compute_direction(direction)
    gscript.run_command('r.viewshed', input=elevation, output=name,
                        coordinates=coords, observer_elevation=1.75,# target_elevation=3,
                        max_distance=limit, direction_range=[mina, maxa],
                        memory=65000, overwrite=True, quiet=True)
    
    # area
    cells = gscript.parse_command('r.univar', map=name, flags='g', quiet=True)['n']
    res = region['nsres']
    area = float(cells) * res * res
    # ndvi
    results = []
    # non_null_cells|null_cells|min|max|range|mean|mean_of_abs|stddev|variance|coeff_var|sum|sum_abs
    for each in sample:
        results.append(gscript.read_command('r.univar', map=name, zones=each, quiet=True, flags='t', separator='comma').strip().splitlines()[1:])

    with open(output, 'w') as f:
#        f.write("id,view_area,z1,z2,z3,z4\n")
        if pid:
            f.write("%s,%.4f,%.4f,%.4f" % (pid, coords[0], coords[1], area))
        else:
            f.write("%.4f,%.4f,%.4f" % (coords[0], coords[1], area))
        for each in results:
            zones = {}
            for line in each:
                zone, label, non_null_cells, null_cells, minim, maxim, range_, mean, mean_of_abs, stddev, variance, coeff_var,  sum_, sum_abs = line.split(',')
                zones[int(zone)] = int(non_null_cells)
            for zone in range(1, 8):
                if zone in zones:
                    area = zones[zone] * res * res
                    f.write(",%.0f" % area)
                else:
                    f.write(",0")
        f.write("\n")
    #gscript.run_command('g.remove', type='raster', name=[name], flags='f', quiet=True)

if __name__ == '__main__':
    options, flags = gscript.parser()
    pid = options['view_id']
    if not pid:
        pid = None
    coords = options['coordinates'].split(',')
    direction = float(options['direction'])
    # coords= 'cycle_track_viewpoints'
    main(elevation='chips_area_dem',
         coords=(float(coords[0]), float(coords[1])),
         output=options['output'],
         pid = pid,
         direction=direction,
         sample=['chips_landuse_7class'])

```

```{python}
import os
filepath="/content/drive/My Drive/People-Environment-Toolbox"
os.chdir(filepath)
!python single_viewshed.py coordinates='132974.298831, 402771.310804' property_id=1 output=test
```

## Viewshed Results


# Download & import CEUS data as GRASS GIS location file 
!wget /content/ "https://raw.githubusercontent.com/gcmillar/CEUS/master/speedinterp_views_dir1_shf.csv"
!wget /content/ "https://raw.githubusercontent.com/gcmillar/CEUS/master/speedinterp_views_dir2_shf.csv"
# head('all_20m_dir2.txt')

## Final Export of Data
> **NOTE**: <img src="https://raw.githubusercontent.com/gcmillar/CEUS/master/img/wait_time.png" alt="runbutton" width="25" height="25">


gs.run_command('v.out.ascii', flags='c', input='chips_cyclists', type='point', output='/content/speedinterp_views_dir2_shf.csv', columns='cat,participant,time,conductance,conductance_z,phasic,phasic_z,tonic,tonic_z,altitude,distance,speed,speed2,power,lon,lat,dist_to,lc_type,dist_to_road,road_type,viewpoint_id_20m')

## Statistical Modeling

The new output from GRASS then of course had to be joined with the output from the parallel python viewshed script. This was carried out in R considering the extensive data manipulation I realized would be necessary. Most important to note is the separation by direction in which cyclists were going. I completely separated the original data analysis file to then appropriately join with the 2 separate files outputted by the python viewshed script. Therefore, I'm attaching two separate csv: stats_direction1, and stats_direction2, each containing the following data:

* participant		   
* time				
* conductance		   
* conductance_z	   
* phasic				
* phasic_z			
* tonic			   
* tonic_z			   
* altitude			
* distance			
* speed			   
* speed2			  
* power			   
* lon				   
* lat				   
* dist_to			   
* lc_type			   
* dist_to_road		
* road_type		   
* viewableLC		  
* buffer50m_lc	  
* buffer100m_lc	   
* buffer200m_lc	   
* buffer500m_lc	 
* direction  
* viewpoint_id_20m	
* viewpoint_id_50m
* viewpoint_id_100m		   
* view_area		   
* landuse_view_area_1
* landuse_view_area_2
* landuse_view_area_3
* landuse_view_area_4

The stats_direction1, and stats_direction2 csvs can technically be joined together if preferred, but they were kept separate throughout for my own peace of mind (and one less thing to worry about).


%%R
# setwd("/Users/garrettmillar/Downloads")
CHIPS_CEUS <- read.csv("/content/CEUS_data.csv", header = TRUE, row.names=NULL, sep="|")
# read in GRASS viewshed results  - both directions (read in as separate files for each viewpoint spacing)
viewshed_20m_dir1 <- read.csv("/content/speedinterp_views_dir1_shf.csv", header = TRUE, row.names=NULL, sep=",")
viewshed_20m_dir2 <- read.csv("/content/speedinterp_views_dir2_shf.csv", header = TRUE, row.names=NULL, sep=",")
head(viewshed_20m_dir2)
CHIPS_waal_til <- subset(CHIPS_CEUS, direction="1")
CHIPS_til_waal <- subset(CHIPS_CEUS, direction="2")
# cleaning viewpoint data - removing points at too high an elevation (e.g., on trees, buildings, etc)
# viewpoints_20m_1 <- read.table("/content/all_20m_dir1.txt", header = F, row.names=NULL, sep=",")
# viewpoints_20m_2 <- read.table("/content/all_20m_dir2.txt", header = F, row.names=NULL, sep=",")

final_stats_20m_url ="https://raw.githubusercontent.com/gcmillar/CEUS/master/chips_all_attr.csv"
# final_stats_20m_url = 'https://raw.githubusercontent.com/gcmillar/CEUS/master/CHIPS_all.csv'
final_stats_20m = pd.read_csv(final_stats_20m_url,error_bad_lines=False)
%load_ext google.colab.data_table
final_stats_20m

%%capture
%unload_ext google.colab.data_table

%%R
dat <- read_csv("/content/chips_all_attr.csv")
head(dat)
table(dat$participant)
table(dat$direction)
names(dat)
datCorrel <- dat[c("participant", "phasic", "landuse_view_area_1_20m","landuse_view_area_2_20m","landuse_view_area_3_20m","landuse_view_area_4_20m", 
                   "landuse_view_area_5_20m", "landuse_view_area_6_20m","landuse_view_area_7_20m","weighted240dist_kmh","dir_range_20m","direction")]
corrgram(datCorrel, order=TRUE, lower.panel=panel.conf,
         upper.panel=panel.pie, text.panel=panel.txt,
         main="landscape predictors")
describe(datCorrel)

%%R
# creating models, including a null model
# lme falsely converges
m0 <- lmer(phasic ~ 1 + (1|participant), data=dat, REML=F)
m1 <- lmer(phasic ~ landuse_view_area_1_20m + landuse_view_area_2_20m + landuse_view_area_3_20m + landuse_view_area_4_20m + 
           landuse_view_area_5_20m + landuse_view_area_6_20m + landuse_view_area_7_20m + (1|participant), data=dat, REML=F)
m2 <- lmer(phasic ~ landuse_view_area_1_20m + landuse_view_area_2_20m + landuse_view_area_3_20m + landuse_view_area_4_20m + 
             landuse_view_area_5_20m + landuse_view_area_6_20m + landuse_view_area_7_20m + weighted240dist_kmh + direction + (1|participant), data=dat, REML=F)
# m3 <- lmer(phasic ~ landuse_view_area_1_20m*weighted240dist_kmh + landuse_view_area_2_20m*weighted240dist_kmh + landuse_view_area_3_20m*weighted240dist_kmh + landuse_view_area_4_20m*weighted240dist_kmh + 
#              landuse_view_area_5_20m*weighted240dist_kmh + landuse_view_area_6_20m*weighted240dist_kmh + landuse_view_area_7_20m*weighted240dist_kmh + weighted240dist_kmh*weighted240dist_kmh + dir + dist_to_road + (1|participant), data=dat.o, REML=F)
# m4 <- lmer(phasic ~ landuse_view_area_1_20m*weighted240dist_kmh + landuse_view_area_2_20m*weighted240dist_kmh + landuse_view_area_3_20m*weighted240dist_kmh + landuse_view_area_4_20m*weighted240dist_kmh + 
#              landuse_view_area_5_20m*weighted240dist_kmh + landuse_view_area_6_20m*weighted240dist_kmh + landuse_view_area_7_20m*weighted240dist_kmh + weighted240dist_kmh*weighted240dist_kmh + dir + dist_to_road + dir_range_20m + (1|participant), data=dat.o, REML=F)
# m2.1 <- lmer(phasic ~ landuse_view_area_1_20m + landuse_view_area_2_20m + landuse_view_area_3_20m + landuse_view_area_4_20m + 
#              landuse_view_area_5_20m + landuse_view_area_6_20m + landuse_view_area_7_20m + weighted240dist_kmh + dir + dir_range_20m + (1|participant), data=dat.o, REML=F)
# # w viewarea
# m2.2 <- lmer(phasic ~ landuse_view_area_1_20m + landuse_view_area_2_20m + landuse_view_area_3_20m + landuse_view_area_4_20m + 
#                landuse_view_area_5_20m + landuse_view_area_6_20m + landuse_view_area_7_20m + weighted240dist_kmh + dir + dist_to_road + dir_range_20m + viewarea + (1|participant), data=dat.o, REML=F)
# comparing models
anova(m0,m1)
anova(m1,m2)
# anova(m2,m3)
# anova(m3,m4)
# anova(m2,m2.1)
# anova(m0,m2)
# anova(m2,m2.1)
# summary(m2.1)
# summary(m2.2) #do not use

<!-- # References

[The Actual Elevation map of the Netherlands (AHN)]<sup>[3](http://ahn.nl) -->